In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import joblib
import time
from glob import glob
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import ee
import ast

In [ ]:
# acz_list = ['Western Himalayan Region', 'Eastern Himalayan Region', 'Lower Gangetic Plain Region',
#             'Middle Gangetic Plain Region', 'Upper Gangetic Plain Region', 'Trans Gangetic Plain Region',
#             'Eastern Plateau & Hills Region', 'Central Plateau & Hills Region', 'Western Plateau and Hills Region',
#             'Southern Plateau and Hills Region', 'East Coast Plains & Hills Region']

acz_list = ['Eastern Plateau & Hills Region']

In [ ]:
best_month_dict = {'Eastern Plateau & Hills Region': 'cc_12',
                   'Middle Gangetic Plain Region': 'cc_10',
                   'Lower Gangetic Plain Region': 'cc_9',
                   'Western Himalayan Region': 'cc_8',
                   'Eastern Himalayan Region': 'cc_10',
                   'Upper Gangetic Plain Region': 'cc_9',
                   'Trans Gangetic Plain Region': 'cc_9',
                   'Central Plateau & Hills Region': 'cc_7',
                   'Western Plateau and Hills Region': 'cc_11',
                   'Southern Plateau and Hills Region': 'cc_8',
                   'East Coast Plains & Hills Region': 'cc_12'}

In [ ]:
# if AY 2023's data is being added, set year = '2023'
year = '2022'

In [ ]:
# We need to correct data from year_2 and year_1
year_1 = int(year)-1
year_2 = int(year)-2
year_3 = int(year)-3
year_4 = int(year)-4

## Data Correction - CCD




In [ ]:
def corrections(df):
    columns = list(df.columns)
    n = len(columns)

    # In case the no. of columns are too less for any corrections to be performed
    if n < 5:
        correction_df = pd.DataFrame(columns=columns)
        return correction_df

    # Correcting the second last year
    correction_df = df[(df[columns[n-1]] == df[columns[n-3]]) & (df[columns[n-3]] == df[columns[n-4]]) & (df[columns[n-2]] != df[columns[n-1]])]
    correction_df.drop_duplicates(inplace=True)

    # Correcting the middle year
    new_df = df[(df[columns[n-5]] == df[columns[n-4]]) & (df[columns[n-4]] == df[columns[n-2]]) & (df[columns[n-2]] == df[columns[n-1]]) & (df[columns[n-3]] != df[columns[n-5]])]
    correction_df = pd.concat([correction_df, new_df], ignore_index=True)
    correction_df.drop_duplicates(inplace=True)
    del(new_df)

    # Actually Performing the corrections once all rows where corrections need to be performed are found
    correction_df.loc[(correction_df[columns[n-1]] == correction_df[columns[n-3]]) & (correction_df[columns[n-3]] == correction_df[columns[n-4]]) & (correction_df[columns[n-2]] != correction_df[columns[n-1]]), columns[n-2]] = correction_df[columns[n-1]]
    correction_df.loc[(correction_df[columns[n-5]] == correction_df[columns[n-4]]) & (correction_df[columns[n-4]] == correction_df[columns[n-2]]) & (correction_df[columns[n-2]] == correction_df[columns[n-1]]) & (correction_df[columns[n-3]] != correction_df[columns[n-5]]), columns[n-3]] = correction_df[columns[n-5]]

    return correction_df

In [ ]:
for agroclimatic_zone in acz_list:
    print(agroclimatic_zone)
    df = pd.read_csv(f'drive/MyDrive/harsh/{agroclimatic_zone}.csv')
    dist_list = list(df['Name'])
    print(f'len(dist_list): {len(dist_list)}')

    path = f'drive/MyDrive/{agroclimatic_zone}/'

    total_corrections = 0
    total_length = 0

    for i in range(len(dist_list)):

        # if i != 11:
        #   continue

        print(i, dist_list[i])
        file_4 = path + dist_list[i] + f"/{year_4}/result_monthly_cc.csv"
        file_3 = path + dist_list[i] + f"/{year_3}/result_monthly_cc.csv"
        file_2 = path + dist_list[i] + f"/{year_2}/result_monthly_cc.csv"
        file_1 = path + dist_list[i] + f"/{year_1}/result_monthly_cc.csv"
        file_0 = path + dist_list[i] + f"/{year}/result_monthly_cc.csv"

        band = best_month_dict[agroclimatic_zone]
        columnList = ['cc_1', 'cc_2', 'cc_3', 'cc_4', 'cc_5', 'cc_6', 'cc_7', 'cc_8', 'cc_9', 'cc_10', 'cc_11', 'cc_12']
        columnList.remove(band)

        try:
            df_4 = pd.read_csv(file_4)
            df_4.drop(columns=columnList, inplace=True)
            df_4.rename(columns={band: 'cover_class'}, inplace=True)

        except Exception as e:
            print(e)
            df_4 = pd.DataFrame(columns=['.geo', 'cover_class'])

        try:
            df_3 = pd.read_csv(file_3)
            df_3.drop(columns=columnList, inplace=True)
            df_3.rename(columns={band: 'cover_class'}, inplace=True)

        except Exception as e:
            print(e)
            df_3 = pd.DataFrame(columns=['.geo', 'cover_class'])

        df_4.rename(columns={'cover_class': f'cc_{year_4}'}, inplace=True)
        df_3.rename(columns={'cover_class': f'cc_{year_3}'}, inplace=True)
        merged_df = pd.merge(df_4, df_3, on='.geo', how='outer')
        del(df_4)
        del(df_3)

        try:
            df_2 = pd.read_csv(file_2)
            df_2.drop(columns=columnList, inplace=True)
            df_2.rename(columns={band: 'cover_class'}, inplace=True)

        except Exception as e:
            print(e)
            df_2 = pd.DataFrame(columns=['.geo', 'cover_class'])

        df_2.rename(columns={'cover_class': f'cc_{year_2}'}, inplace=True)
        merged_df = pd.merge(merged_df, df_2, on='.geo', how='outer')
        del(df_2)

        try:
            df_1 = pd.read_csv(file_1)
            df_1.drop(columns=columnList, inplace=True)
            df_1.rename(columns={band: 'cover_class'}, inplace=True)
        except Exception as e:
            print(e)
            df_1 = pd.DataFrame(columns=['.geo', 'cover_class'])

        df_1.rename(columns={'cover_class': f'cc_{year_1}'}, inplace=True)
        merged_df = pd.merge(merged_df, df_1, on='.geo', how='outer')
        del(df_1)

        try:
            df_0 = pd.read_csv(file_0)
            df_0.drop(columns=columnList, inplace=True)
            df_0.rename(columns={band: 'cover_class'}, inplace=True)

        except Exception as e:
            print(e)
            df_0 = pd.DataFrame(columns=['.geo', 'cover_class'])

        df_0.rename(columns={'cover_class': f'cc_{year}'}, inplace=True)
        merged_df = pd.merge(merged_df, df_0, on='.geo', how='outer')
        del(df_0)


        merged_df = merged_df[['.geo', f'cc_{year_4}', f'cc_{year_3}', f'cc_{year_2}', f'cc_{year_1}', f'cc_{year}']]
        total_length += len(merged_df)
        print("Length of merged_df:", len(merged_df))
        print("Total Length:", total_length)
        correction_df = corrections(merged_df)
        del(merged_df)
        total_corrections += len(correction_df)
        print(f'Correction_df length: {len(correction_df)}')
        print(f'Total Corrections: {total_corrections}')
        if len(correction_df) > 0:
            correction_year_2 = correction_df[['.geo', f'cc_{year_2}']]
            correction_year_1 = correction_df[['.geo', f'cc_{year_1}']]
            correction_year_2.to_csv(f'{path}{dist_list[i]}/{year_2}/result_monthly_cc_corrections.csv', index=False)
            correction_year_1.to_csv(f'{path}{dist_list[i]}/{year_1}/result_monthly_cc_corrections.csv', index=False)

        del(correction_df)

## Data Correction - CH

In [ ]:
# Column Order
# ['.geo',
# 'rh98_{year_4}', 'rh98_{year_3}', 'rh98_{year_2}', 'rh98_{year_1}', 'rh98_{year}',
# 'rh75_{year_4}', 'rh75_{year_3}', 'rh75_{year_2}', 'rh75_{year_1}', 'rh75_{year}',
# 'rh50_{year_4}', 'rh50_{year_3}', 'rh50_{year_2}', 'rh50_{year_1}', 'rh50_{year}']


def corrections(df):
    columns = list(df.columns)
    n = len(columns)

    # Correcting the second last year - rh98
    correction_df = df[(df[columns[5]] == df[columns[3]]) & (df[columns[3]] == df[columns[2]]) & (df[columns[4]] != df[columns[5]])]
    correction_df.drop_duplicates(inplace=True)

    # Correcting the middle year - rh98
    new_df = df[(df[columns[1]] == df[columns[2]]) & (df[columns[2]] == df[columns[4]]) & (df[columns[4]] == df[columns[5]]) & (df[columns[3]] != df[columns[1]])]
    correction_df = pd.concat([correction_df, new_df], ignore_index=True)
    correction_df.drop_duplicates(inplace=True)
    del(new_df)


    # Correcting the second last year - rh75
    new_df = df[(df[columns[10]] == df[columns[8]]) & (df[columns[8]] == df[columns[7]]) & (df[columns[9]] != df[columns[10]])]
    correction_df = pd.concat([correction_df, new_df], ignore_index=True)
    del(new_df)
    correction_df.drop_duplicates(inplace=True)

    # Correcting the middle year - rh75
    new_df = df[(df[columns[6]] == df[columns[7]]) & (df[columns[7]] == df[columns[9]]) & (df[columns[9]] == df[columns[10]]) & (df[columns[8]] != df[columns[6]])]
    correction_df = pd.concat([correction_df, new_df], ignore_index=True)
    correction_df.drop_duplicates(inplace=True)
    del(new_df)


    # Correcting the second last year - rh50
    new_df = df[(df[columns[15]] == df[columns[13]]) & (df[columns[13]] == df[columns[12]]) & (df[columns[14]] != df[columns[15]])]
    correction_df = pd.concat([correction_df, new_df], ignore_index=True)
    del(new_df)
    correction_df.drop_duplicates(inplace=True)

    # Correcting the middle year - rh50
    new_df = df[(df[columns[11]] == df[columns[12]]) & (df[columns[12]] == df[columns[14]]) & (df[columns[14]] == df[columns[15]]) & (df[columns[13]] != df[columns[11]])]
    correction_df = pd.concat([correction_df, new_df], ignore_index=True)
    correction_df.drop_duplicates(inplace=True)
    del(new_df)


    # Actually Performing the corrections once all rows where corrections need to be performed are found

    # rh98
    correction_df.loc[(correction_df[columns[5]] == correction_df[columns[3]]) & (correction_df[columns[3]] == correction_df[columns[2]]) & (correction_df[columns[4]] != correction_df[columns[5]]), columns[4]] = correction_df[columns[5]]
    correction_df.loc[(correction_df[columns[1]] == correction_df[columns[2]]) & (correction_df[columns[2]] == correction_df[columns[4]]) & (correction_df[columns[4]] == correction_df[columns[5]]) & (correction_df[columns[3]] != correction_df[columns[1]]), columns[3]] = correction_df[columns[1]]


    # rh75
    correction_df.loc[(correction_df[columns[10]] == correction_df[columns[8]]) & (correction_df[columns[8]] == correction_df[columns[7]]) & (correction_df[columns[9]] != correction_df[columns[10]]), columns[9]] = correction_df[columns[10]]
    correction_df.loc[(correction_df[columns[6]] == correction_df[columns[7]]) & (correction_df[columns[7]] == correction_df[columns[9]]) & (correction_df[columns[9]] == correction_df[columns[10]]) & (correction_df[columns[8]] != correction_df[columns[6]]), columns[8]] = correction_df[columns[6]]


    # rh50
    correction_df.loc[(correction_df[columns[15]] == correction_df[columns[13]]) & (correction_df[columns[13]] == correction_df[columns[12]]) & (correction_df[columns[14]] != correction_df[columns[15]]), columns[14]] = correction_df[columns[15]]
    correction_df.loc[(correction_df[columns[11]] == correction_df[columns[12]]) & (correction_df[columns[12]] == correction_df[columns[14]]) & (correction_df[columns[14]] == correction_df[columns[15]]) & (correction_df[columns[13]] != correction_df[columns[11]]), columns[13]] = correction_df[columns[11]]

    return correction_df

In [ ]:
# Function to convert string representation of list to an actual list
def convert_to_list(string):
    return ast.literal_eval(string)


for agroclimatic_zone in acz_list:
    print(agroclimatic_zone)

    df = pd.read_csv('drive/MyDrive/harsh/district_to_agroclimaticZone_mapping.csv')
    df['IntersectingZones'] = df['IntersectingZones'].apply(convert_to_list)
    district_mapping_df = df[df['AgroclimaticZone'] == agroclimatic_zone][['District', 'IntersectingZones']]
    dist_list = []
    for ind in district_mapping_df.index:
        district = district_mapping_df.loc[ind, 'District']
        zones = district_mapping_df['IntersectingZones'][ind]
        dist_list.append(district)


    print(f'len(dist_list): {len(dist_list)}')

    path = f'drive/MyDrive/{agroclimatic_zone}/'

    total_corrections = 0
    total_length = 0

    for i in range(len(dist_list)):

        # if i != 11:
        #   continue

        print(i, dist_list[i])
        file_4 = path + dist_list[i] + f"/{year_4}/result_chm.csv"
        file_3 = path + dist_list[i] + f"/{year_3}/result_chm.csv"
        file_2 = path + dist_list[i] + f"/{year_2}/result_chm.csv"
        file_1 = path + dist_list[i] + f"/{year_1}/result_chm.csv"
        file_0 = path + dist_list[i] + f"/{year}/result_chm.csv"

        try:
            df_4 = pd.read_csv(file_4)
            df_4.rename(columns={'rh98_class': f'rh98_{year_4}', 'rh75_class': f'rh75_{year_4}', 'rh50_class': f'rh50_{year_4}'}, inplace=True)

        except Exception as e:
            print(e)
            df_4 = pd.DataFrame(columns=['.geo', f'rh98_{year_4}', f'rh75_{year_4}', f'rh50_{year_4}', f'ch_{year_4}'])

        try:
            df_3 = pd.read_csv(file_3)
            df_3.rename(columns={'rh98_class': f'rh98_{year_3}', 'rh75_class': f'rh75_{year_3}', 'rh50_class': f'rh50_{year_3}'}, inplace=True)

        except Exception as e:
            print(e)
            df_3 = pd.DataFrame(columns=['.geo', f'rh98_{year_3}', f'rh75_{year_3}', f'rh50_{year_3}', f'ch_{year_3}'])

        merged_df = pd.merge(df_4, df_3, on='.geo', how='outer')
        del(df_4)
        del(df_3)

        try:
            df_2 = pd.read_csv(file_2)
            df_2.rename(columns={'rh98_class': f'rh98_{year_2}', 'rh75_class': f'rh75_{year_2}', 'rh50_class': f'rh50_{year_2}'}, inplace=True)

        except Exception as e:
            print(e)
            df_2 = pd.DataFrame(columns=['.geo', f'rh98_{year_2}', f'rh75_{year_2}', f'rh50_{year_2}', f'ch_{year_2}'])

        merged_df = pd.merge(merged_df, df_2, on='.geo', how='outer')
        del(df_2)

        try:
            df_1 = pd.read_csv(file_1)
            df_1.rename(columns={'rh98_class': f'rh98_{year_1}', 'rh75_class': f'rh75_{year_1}', 'rh50_class': f'rh50_{year_1}'}, inplace=True)
        except Exception as e:
            print(e)
            df_1 = pd.DataFrame(columns=['.geo', f'rh98_{year_1}', f'rh75_{year_1}', f'rh50_{year_1}', f'ch_{year_1}'])

        merged_df = pd.merge(merged_df, df_1, on='.geo', how='outer')
        del(df_1)

        try:
            df_0 = pd.read_csv(file_0)
            df_0.rename(columns={'rh98_class': f'rh98_{year}', 'rh75_class': f'rh75_{year}', 'rh50_class': f'rh50_{year}'}, inplace=True)

        except Exception as e:
            print(e)
            df_0 = pd.DataFrame(columns=['.geo', f'rh98_{year}', f'rh75_{year}', f'rh50_{year}', f'ch_{year}'])

        merged_df = pd.merge(merged_df, df_0, on='.geo', how='outer')
        del(df_0)

        merged_df = merged_df[['.geo', f'rh98_{year_4}', f'rh98_{year_3}', f'rh98_{year_2}', f'rh98_{year_1}', f'rh98_{year}', f'rh75_{year_4}', f'rh75_{year_3}', f'rh75_{year_2}', f'rh75_{year_1}', f'rh75_{year}', f'rh50_{year_4}', f'rh50_{year_3}', f'rh50_{year_2}', f'rh50_{year_1}', f'rh50_{year}']]
        total_length += len(merged_df)
        print("Length of merged_df:", len(merged_df))
        print("Total Length:", total_length)
        correction_df = corrections(merged_df)
        del(merged_df)
        total_corrections += len(correction_df)
        print(f'Correction_df length: {len(correction_df)}')
        print(f'Total Corrections: {total_corrections}')
        if len(correction_df) > 0:

            choices = [0, 0, 1, 2, 1, 2]


            conditions = [
                (correction_df[f'rh50_{year_4}'] == 0) & (correction_df[f'rh75_{year_4}'] == 0) & (correction_df[f'rh98_{year_4}'] == 0),
                (correction_df[f'rh50_{year_4}'] == 0) & (correction_df[f'rh75_{year_4}'] == 0) & (correction_df[f'rh98_{year_4}'] == 1),
                (correction_df[f'rh50_{year_4}'] == 0) & (correction_df[f'rh75_{year_4}'] == 1) & (correction_df[f'rh98_{year_4}'] == 0),
                (correction_df[f'rh50_{year_4}'] == 0) & (correction_df[f'rh75_{year_4}'] == 1) & (correction_df[f'rh98_{year_4}'] == 1),
                (correction_df[f'rh50_{year_4}'] == 1) & (correction_df[f'rh75_{year_4}'] == 0) & (correction_df[f'rh98_{year_4}'] == 0),
                (correction_df[f'rh50_{year_4}'] == 1) & (correction_df[f'rh75_{year_4}'] == 0) & (correction_df[f'rh98_{year_4}'] == 1)
            ]
            correction_df[f'ch_{year_4}'] = np.select(conditions, choices, default=3)


            conditions = [
                (correction_df[f'rh50_{year_3}'] == 0) & (correction_df[f'rh75_{year_3}'] == 0) & (correction_df[f'rh98_{year_3}'] == 0),
                (correction_df[f'rh50_{year_3}'] == 0) & (correction_df[f'rh75_{year_3}'] == 0) & (correction_df[f'rh98_{year_3}'] == 1),
                (correction_df[f'rh50_{year_3}'] == 0) & (correction_df[f'rh75_{year_3}'] == 1) & (correction_df[f'rh98_{year_3}'] == 0),
                (correction_df[f'rh50_{year_3}'] == 0) & (correction_df[f'rh75_{year_3}'] == 1) & (correction_df[f'rh98_{year_3}'] == 1),
                (correction_df[f'rh50_{year_3}'] == 1) & (correction_df[f'rh75_{year_3}'] == 0) & (correction_df[f'rh98_{year_3}'] == 0),
                (correction_df[f'rh50_{year_3}'] == 1) & (correction_df[f'rh75_{year_3}'] == 0) & (correction_df[f'rh98_{year_3}'] == 1)
            ]
            correction_df[f'ch_{year_3}'] = np.select(conditions, choices, default=3)


            conditions = [
                (correction_df[f'rh50_{year_2}'] == 0) & (correction_df[f'rh75_{year_2}'] == 0) & (correction_df[f'rh98_{year_2}'] == 0),
                (correction_df[f'rh50_{year_2}'] == 0) & (correction_df[f'rh75_{year_2}'] == 0) & (correction_df[f'rh98_{year_2}'] == 1),
                (correction_df[f'rh50_{year_2}'] == 0) & (correction_df[f'rh75_{year_2}'] == 1) & (correction_df[f'rh98_{year_2}'] == 0),
                (correction_df[f'rh50_{year_2}'] == 0) & (correction_df[f'rh75_{year_2}'] == 1) & (correction_df[f'rh98_{year_2}'] == 1),
                (correction_df[f'rh50_{year_2}'] == 1) & (correction_df[f'rh75_{year_2}'] == 0) & (correction_df[f'rh98_{year_2}'] == 0),
                (correction_df[f'rh50_{year_2}'] == 1) & (correction_df[f'rh75_{year_2}'] == 0) & (correction_df[f'rh98_{year_2}'] == 1)
            ]
            correction_df[f'ch_{year_2}'] = np.select(conditions, choices, default=3)


            conditions = [
                (correction_df[f'rh50_{year_1}'] == 0) & (correction_df[f'rh75_{year_1}'] == 0) & (correction_df[f'rh98_{year_1}'] == 0),
                (correction_df[f'rh50_{year_1}'] == 0) & (correction_df[f'rh75_{year_1}'] == 0) & (correction_df[f'rh98_{year_1}'] == 1),
                (correction_df[f'rh50_{year_1}'] == 0) & (correction_df[f'rh75_{year_1}'] == 1) & (correction_df[f'rh98_{year_1}'] == 0),
                (correction_df[f'rh50_{year_1}'] == 0) & (correction_df[f'rh75_{year_1}'] == 1) & (correction_df[f'rh98_{year_1}'] == 1),
                (correction_df[f'rh50_{year_1}'] == 1) & (correction_df[f'rh75_{year_1}'] == 0) & (correction_df[f'rh98_{year_1}'] == 0),
                (correction_df[f'rh50_{year_1}'] == 1) & (correction_df[f'rh75_{year_1}'] == 0) & (correction_df[f'rh98_{year_1}'] == 1)
            ]
            correction_df[f'ch_{year_1}'] = np.select(conditions, choices, default=3)


            conditions = [
                (correction_df[f'rh50_{year}'] == 0) & (correction_df[f'rh75_{year}'] == 0) & (correction_df[f'rh98_{year}'] == 0),
                (correction_df[f'rh50_{year}'] == 0) & (correction_df[f'rh75_{year}'] == 0) & (correction_df[f'rh98_{year}'] == 1),
                (correction_df[f'rh50_{year}'] == 0) & (correction_df[f'rh75_{year}'] == 1) & (correction_df[f'rh98_{year}'] == 0),
                (correction_df[f'rh50_{year}'] == 0) & (correction_df[f'rh75_{year}'] == 1) & (correction_df[f'rh98_{year}'] == 1),
                (correction_df[f'rh50_{year}'] == 1) & (correction_df[f'rh75_{year}'] == 0) & (correction_df[f'rh98_{year}'] == 0),
                (correction_df[f'rh50_{year}'] == 1) & (correction_df[f'rh75_{year}'] == 0) & (correction_df[f'rh98_{year}'] == 1)
            ]
            correction_df[f'ch_{year}'] = np.select(conditions, choices, default=3)

            cols = correction_df.columns
            for j in range(1, len(cols)):
                correction_df[cols[j]] = correction_df[cols[j]].astype('Int64')

            correction_year_2 = correction_df[['.geo', f'rh50_{year_2}', f'rh75_{year_2}', f'rh98_{year_2}', f'ch_{year_2}']]
            correction_year_1 = correction_df[['.geo', f'rh50_{year_1}', f'rh75_{year_1}', f'rh98_{year_1}', f'ch_{year_1}']]
            correction_year_2.to_csv(f'{path}{dist_list[i]}/{year_2}/result_chm_corrections.csv', index=False)
            correction_year_1.to_csv(f'{path}{dist_list[i]}/{year_1}/result_chm_corrections.csv', index=False)

        del(correction_df)